In [ ]:
from transformers import pipeline
import torch
from PIL import Image
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from lac.util import load_data
import lac.params as params

%load_ext autoreload
%autoreload 2

## Depth-Anything from HuggingFace


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
checkpoint = "depth-anything/Depth-Anything-V2-base-hf"
pipe = pipeline("depth-estimation", model=checkpoint, device=device)

In [ ]:
data_path = Path("../../output/NavAgent/map1_preset4_gtnav_steer")
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)

In [ ]:
i = 1600

# image_path = os.path.expanduser(
#     "~/LunarAutonomyChallenge/output/Old/data_collection_1/front_left/{}.png".format(i)
# )
# image = Image.open(image_path)
image = Image.open(data_path / "FrontLeft" / f"{i}.png")

predictions = pipe(image)

# Plot image and predicted depth side by side
fig, axes = plt.subplots(1, 2, figsize=(20, 10), dpi=300, gridspec_kw={"wspace": 0, "hspace": 0})
axes[0].imshow(image, cmap="gray")
axes[1].imshow(predictions["depth"], cmap="gray")
for ax in axes:
    ax.axis("off")
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 5), dpi=100)
plt.imshow(predictions["depth"], cmap="gray")
plt.axis("off")
plt.show()

## Apple Depth Pro


In [ ]:
import depth_pro

In [ ]:
# Load model and preprocessing transform
model, transform = depth_pro.create_model_and_transforms(device=device)
model.eval()

# Load and preprocess an image.
image, _, f_px = depth_pro.load_rgb(image_path)
image = transform(image)

# Run inference.
prediction = model.infer(image, f_px=f_px)
depth = prediction["depth"]  # Depth in [m].
focallength_px = prediction["focallength_px"]  # Focal length in pixels.

In [ ]:
plt.imshow(depth.cpu(), cmap="gray")

## Stereo

- Stereo baseline = 0.162 m
- All cameras horizontal FOV = 1.22 radians (70 degrees)

"The cameras are modelled as perfect pinhole cameras with square pixels, there is no lens distortion. Lens flare from the sun is modelled, this should be considered as a potential source of error in segmentation and feature detection. Each camera has the same field of view of 1.22 radians (70 degrees). The resolution is set by the agent upon initialization in the sensors() method. The maximum resolution allowed is 2448 x 2048 pixels, if a resolution higher than this is requested the resolution will be clipped to the maximum and a warning will be given on the command line."

From discord:
"Effectively, there is no focal length because the simulator does not model a physical camera, it is modelled as a perfect pinhole camera. Normally, the focal length is given in mm and to relate a pixel coordinate to a line extending from the camera center into the world, you need to use the pixel dimensions in mm.

The trick is to express the focal length in terms of pixels. Draw the camera geometry in a diagram, the focal length is the distance between the camera center and the image plane. You also know the width and height of the sensor in terms of pixels because you set this in your agent set up, you also know the FOV of the camera. Using triangular geometric relations you can express the focal length in terms of pixels."


In [ ]:
W, H = 1280, 720
FOV = 1.22  # radians
BASELINE = 0.162  # meters

data_path = Path("../../output/data_collection_1")

In [ ]:
i = 40

left_image_path = data_path / "front_left" / f"{i}.png"
right_image_path = data_path / "front_right" / f"{i}.png"
left_image = Image.open(left_image_path)
right_image = Image.open(data_path / "front_right" / f"{i}.png")

# Plot image and predicted depth side by side
fig, axes = plt.subplots(1, 2, figsize=(20, 10), gridspec_kw={"wspace": 0, "hspace": 0})
axes[0].imshow(left_image, cmap="gray")
axes[1].imshow(right_image, cmap="gray")
for ax in axes:
    ax.axis("off")
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

### OpenCV


In [ ]:
from lac.perception.depth import compute_stereo_depth

In [ ]:
focal_length_x = W / (2 * np.tan(FOV / 2))
focal_length_y = H / (2 * np.tan(FOV / 2))

disparity, depth = compute_stereo_depth(
    np.array(left_image), np.array(right_image), BASELINE, focal_length_x, semi_global=False
)

In [ ]:
print(focal_length_x, focal_length_y)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10), gridspec_kw={"wspace": 0, "hspace": 0})
axes[0].imshow(disparity, cmap="gray")
axes[1].imshow(depth, cmap="gray")
for ax in axes:
    ax.axis("off")
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

## LightGlue matching


In [ ]:
from lightglue import LightGlue, SuperPoint, DISK, SIFT, ALIKED, DoGHardNet
from lightglue.utils import load_image, rbd
from lightglue import match_pair, viz2d

from lac.params import FL_X, STEREO_BASELINE
from lac.util import load_data

In [ ]:
data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/NavAgent/map1_preset4_gtnav_steer")
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)

i = 100
I1_path = os.path.join(data_path, "FrontLeft", f"{i}.png")
I2_path = os.path.join(data_path, "FrontRight", f"{i}.png")
I1 = cv.imread(I1_path, cv.IMREAD_GRAYSCALE)
I2 = cv.imread(I2_path, cv.IMREAD_GRAYSCALE)

# Plot images side by side
fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].imshow(I1, cmap="gray")
ax[1].imshow(I2, cmap="gray")
plt.show()

In [ ]:
def grayscale_to_3ch_tensor(np_image):
    # Ensure the input is float32 (or float64 if needed)
    np_image = np_image.astype(np.float32) / 255.0 if np_image.max() > 1 else np_image
    # Add channel dimension and repeat across 3 channels
    torch_tensor = torch.from_numpy(np_image).unsqueeze(0).repeat(3, 1, 1)
    return torch_tensor

In [ ]:
# SuperPoint+LightGlue
extractor = SuperPoint(max_num_keypoints=2048).eval().cuda()  # load the extractor
matcher = LightGlue(features="superpoint").eval().cuda()  # load the matcher

# load each image as a torch.Tensor on GPU with shape (3,H,W), normalized in [0,1]
image0 = grayscale_to_3ch_tensor(I1).cuda()
image1 = grayscale_to_3ch_tensor(I2).cuda()

feats0, feats1, matches01 = match_pair(extractor, matcher, image0, image1)
matches = matches01["matches"]  # indices with shape (K,2)
points0 = feats0["keypoints"][matches[..., 0]]  # coordinates in image #0, shape (K,2)
points1 = feats1["keypoints"][matches[..., 1]]  # coordinates in image #1, shape (K,2)

In [ ]:
disparities = (points0 - points1)[:, 0]
depths = FL_X * STEREO_BASELINE / disparities

In [ ]:
# Plot depths on top of left image
fig, ax = plt.subplots(1, 1, figsize=(15, 15))
ax.imshow(I1, cmap="gray")
ax.scatter(points0[:, 0], points0[:, 1], c=depths, cmap="viridis", s=5)
plt.show()

In [ ]:
from lac.perception.vision import project_pixel_to_3D
from lac.params import CAMERA_INTRINSICS
from lac.utils.plotting import plot_3d_points, plot_surface
from lac.utils.frames import opencv_to_camera, get_cam_pose_rover, apply_transform

In [ ]:
CAM_TO_ROVER = get_cam_pose_rover("FrontLeft")
rover_pose = poses[i]
world_points = []

for point, depth in zip(points0, depths):
    point_opencv = project_pixel_to_3D(point, depth, CAMERA_INTRINSICS)
    point_cam = opencv_to_camera(point_opencv)
    point_rover = apply_transform(CAM_TO_ROVER, point_cam)
    point_world = apply_transform(rover_pose, point_rover)
    world_points.append(point_world)

world_points = np.array(world_points)

In [ ]:
map = np.load("../../data/heightmaps/competition/Moon_Map_01_preset_0.dat", allow_pickle=True)

In [ ]:
fig = plot_surface(map)
fig = plot_3d_points(world_points, color="red", markersize=3, fig=fig)
fig.update_layout(width=1600, height=900, scene_aspectmode="data")
fig.show()

In [ ]:
fig.write_html("lightglue_stereo.html")

In [ ]:
axes = viz2d.plot_images([image0, image1])
viz2d.plot_matches(points0, points1, color="lime", lw=0.2)
viz2d.add_text(0, f"Stop after {matches01['stop']} layers", fs=20)

kpc0, kpc1 = viz2d.cm_prune(matches01["prune0"]), viz2d.cm_prune(matches01["prune1"])
viz2d.plot_images([image0, image1])
viz2d.plot_keypoints([feats0["keypoints"], feats1["keypoints"]], colors=[kpc0, kpc1], ps=10)

# Stereo with segmentation


In [ ]:
from lac.util import load_data
from lac.utils.visualization import overlay_mask
from lac.perception.segmentation import Segmentation
from lac.perception.vision import project_pixel_to_3D

In [ ]:
segmentation = Segmentation()

In [ ]:
data_path = Path("../../output/nav_agent_preset_2_seed_4")
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)

In [ ]:
i = 1020

left_image = Image.open(data_path / "FrontLeft" / f"{i}.png")
right_image = Image.open(data_path / "FrontRight" / f"{i}.png")

left_seg_results, left_seg_mask = segmentation.segment_rocks(left_image.convert("RGB"))
right_seg_results, right_seg_mask = segmentation.segment_rocks(right_image.convert("RGB"))

left_overlay = overlay_mask(np.array(left_image), left_seg_mask)
right_overlay = overlay_mask(np.array(right_image), right_seg_mask)

# Plot image and predicted depth side by side
fig, axes = plt.subplots(1, 2, figsize=(20, 10), gridspec_kw={"wspace": 0, "hspace": 0})
axes[0].imshow(left_overlay)
axes[1].imshow(right_overlay)
for ax in axes:
    ax.axis("off")
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

In [ ]:
from lac.perception.depth import stereo_depth_from_segmentation, project_depths_to_world
from lac.utils.visualization import overlay_stereo_rock_depths

results = stereo_depth_from_segmentation(
    left_seg_results, right_seg_results, params.STEREO_BASELINE, params.FL_X
)
left_overlay = overlay_stereo_rock_depths(left_overlay, results)
plt.imshow(left_overlay)

In [ ]:
for result in results:
    point = project_pixel_to_3D(result["left_centroid"], result["depth"], params.CAMERA_INTRINSICS)
    print(point)

In [ ]:
rover_pose = poses[i]

project_depths_to_world(results, rover_pose, cam_name="FrontLeft", cam_config=cam_config)

We can also have a matching method based on the masks themselves, i.e., for each predicted mask in left image,
search for the most similar mask in the right image.


# Heightmap reprojection


In [ ]:
heightmap_path = "../../data/heightmaps/Moon_Map_01_0_rep0.dat"
heightmap = np.load(heightmap_path, allow_pickle=True)
heightmap.shape

In [ ]:
camera_pose = np.eye(4)